# SBTi-Finance Tool - Portfolio Aggregation
In this notebook we'll give some examples on how the portfolio aggregation methods can be used.

Please see the [methodology](https://sciencebasedtargets.org/wp-content/uploads/2020/09/Temperature-Rating-Methodology-V1.pdf), [guidance](https://sciencebasedtargets.org/wp-content/uploads/2020/10/Financial-Sector-Science-Based-Targets-Guidance-Pilot-Version.pdf) and the [technical documentation](https://sciencebasedtargets.github.io/SBTi-finance-tool/)  for more details on the different aggregation methods.

See 1_analysis_example (on [Colab](https://colab.research.google.com/github/ScienceBasedTargets/SBTi-finance-tool/blob/main/examples/1_analysis_example.ipynb) or [Github](https://github.com/ScienceBasedTargets/SBTi-finance-tool/blob/main/examples/1_analysis_example.ipynb)) for more in depth example of how to work with Jupyter Notebooks in general and SBTi notebooks in particular. 


## Setting up
First we will set up the imports, data providers, and load the portfolio. 

For more examples of this process, please refer to notebook 1 & 2 (analysis and quick calculation example).


In [ ]:
%pip install sbti-finance-tool

In [ ]:
%load_ext autoreload
%autoreload 2
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario, ScenarioType, EngagementType
from SBTi.target_validation import TargetProtocol
from SBTi.interfaces import ETimeFrames, EScope
%aimport -pandas
import pandas as pd

In [ ]:
# Download the dummy data
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/data_provider_example.xlsx"):
    urllib.request.urlretrieve("https://github.com/ScienceBasedTargets/SBTi-finance-tool/raw/main/examples/data/data_provider_example.xlsx", "data/data_provider_example.xlsx")
if not os.path.isfile("data/example_portfolio.csv"):
    urllib.request.urlretrieve("https://github.com/ScienceBasedTargets/SBTi-finance-tool/raw/main/examples/data/example_portfolio.csv", "data/example_portfolio.csv")


In [ ]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")
df_portfolio = pd.read_csv("data/example_portfolio.csv", encoding="iso-8859-1")

# Print original columns to diagnose
print("Original columns:", df_portfolio.columns.tolist())

# Create a more cautious renaming dictionary based on what columns exist
rename_dict = {}
if "Company Name" in df_portfolio.columns:
    rename_dict["Company Name"] = "company_name"
if "ISIN" in df_portfolio.columns:
    rename_dict["ISIN"] = "isin" 
if "LEI" in df_portfolio.columns:
    rename_dict["LEI"] = "lei"
if "Sector" in df_portfolio.columns:
    rename_dict["Sector"] = "sector"
if "Target" in df_portfolio.columns:
    rename_dict["Target"] = "full_target_language"
if "Net-Zero Committed" in df_portfolio.columns:
    rename_dict["Net-Zero Committed"] = "net_zero_status"
if "Near term - Target Status" in df_portfolio.columns:
    rename_dict["Near term - Target Status"] = "near_term_status"
if "Target Classification" in df_portfolio.columns:
    rename_dict["Target Classification"] = "target_classification_long"
if "Extension" in df_portfolio.columns:
    rename_dict["Extension"] = "reason_for_extension_or_removal"
if "Date" in df_portfolio.columns:
    rename_dict["Date"] = "date_updated"

# Apply column renaming
df_portfolio = df_portfolio.rename(columns=rename_dict)

# Handle legacy column names if they exist
if 'company_isin' in df_portfolio.columns:
    df_portfolio.rename(columns={'company_isin': 'isin'}, inplace=True)
if 'company_lei' in df_portfolio.columns:
    df_portfolio.rename(columns={'company_lei': 'lei'}, inplace=True)

# Ensure required columns exist
required_columns = ['company_id', 'company_name', 'isin', 'lei', 'investment_value']
for col in required_columns:
    if col not in df_portfolio.columns:
        df_portfolio[col] = None

# Convert identifiers to string
if 'isin' in df_portfolio.columns:
    df_portfolio['isin'] = df_portfolio['isin'].astype(str)
if 'lei' in df_portfolio.columns:
    df_portfolio['lei'] = df_portfolio['lei'].astype(str)

# Check for duplicate values in the 'company_id' column
duplicate_ids = df_portfolio[df_portfolio.duplicated('company_id', keep=False)]
if not duplicate_ids.empty:
    print("Error: Duplicate values found in the 'company_id' column:")
    print(duplicate_ids)
else:
    print("No duplicate values found in the 'company_id' column.")

# Display final columns to verify
print("Final columns:", df_portfolio.columns.tolist())

# Convert to portfolio objects and get data
companies = SBTi.utils.dataframe_to_portfolio(df_portfolio)
try:
    portfolio_data = SBTi.utils.get_data([provider], companies)
    scenarios = {}
    print("Successfully loaded portfolio data")
except Exception as e:
    print(f"Error loading portfolio data: {str(e)}")
    # Add more error handling if needed
    import traceback
    traceback.print_exc()

In [ ]:
temperature_score = TemperatureScore(time_frames=list(SBTi.interfaces.ETimeFrames), scopes=[EScope.S1S2, EScope.S3, EScope.S1S2S3])
amended_portfolio = temperature_score.calculate(data_providers=[provider], portfolio=companies)

## Calculate the aggregated temperature score
Calculate an aggregated temperature score. This can be done using different aggregation methods. The termperature scores are calculated per time-frame/scope combination.

### WATS
Weighted Average Temperature Score (WATS): Temperature scores are allocated based on portfolio weights.
This method uses the "investment_value" field to be defined in your portfolio data.

In [ ]:
temperature_score.aggregation_method = PortfolioAggregationMethod.WATS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
df_wats = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'WATS': df_wats})
df_wats

### TETS
Total emissions weighted temperature score (TETS): Temperature scores are allocated based on historical emission weights using total company emissions. 
In addition to the portfolios "investment value" the TETS method requires company emissions, please refer to [Data Legends - Fundamental Data](https://ofbdabv.github.io/SBTi/Legends.html#fundamental-data) for more details

In [ ]:
temperature_score.aggregation_method = PortfolioAggregationMethod.TETS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
df_tets = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'TETS': df_tets})
df_tets

### MOTS
Market Owned emissions weighted temperature score (MOTS): Temperature scores are allocated based on an equity ownership approach.
In addition to the portfolios "investment value" the MOTS method requires company emissions and market cap, please refer to  [Data Legends - Fundamental Data](https://ofbdabv.github.io/SBTi/Legends.html#fundamental-data) for more details

In [ ]:
temperature_score.aggregation_method = PortfolioAggregationMethod.MOTS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
df_mots = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'MOTS': df_mots})
df_mots

### EOTS
Enterprise Owned emissions weighted temperature score (EOTS): Temperature scores are allocated based
on an enterprise ownership approach. 
In addition to the portfolios "investment value" the EOTS method requires company emissions and enterprise value, please refer to  [Data Legends - Fundamental Data](https://ofbdabv.github.io/SBTi/Legends.html#fundamental-data) for more details

In [ ]:
temperature_score.aggregation_method = PortfolioAggregationMethod.EOTS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
df_eots = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'EOTS': df_eots})
df_eots

### ECOTS
Enterprise Value + Cash emissions weighted temperature score (ECOTS): Temperature scores are allocated based on an enterprise value (EV) plus cash & equivalents ownership approach. 
In addition to the portfolios "investment value" the ECOTS method requires company emissions, company cash equivalents and enterprise value; please refer to  [Data Legends - Fundamental Data](https://sciencebasedtargets.github.io/SBTi-finance-tool/Legends.html#fundamental-data) for more details

In [ ]:
temperature_score.aggregation_method = PortfolioAggregationMethod.ECOTS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
df_ecots = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'ECOTS': df_ecots})
df_ecots

### AOTS
Total Assets emissions weighted temperature score (AOTS): Temperature scores are allocated based on a total assets ownership approach. 
In addition to the portfolios "investment value" the AOTS method requires company emissions and company total assets; please refer to  [Data Legends - Fundamental Data](https://sciencebasedtargets.github.io/SBTi-finance-tool/Legends.html#fundamental-data) for more details

In [ ]:
temperature_score.aggregation_method = PortfolioAggregationMethod.AOTS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
df_aots = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'AOTS': df_aots})
df_aots

### ROTS
Revenue owned emissions weighted temperature score (ROTS): Temperature scores are allocated based on the share of revenue.
In addition to the portfolios "investment value" the ROTS method requires company emissions and company revenue; please refer to  [Data Legends - Fundamental Data](https://sciencebasedtargets.github.io/SBTi-finance-tool/Legends.html#fundamental-data) for more details

In [ ]:
temperature_score.aggregation_method = PortfolioAggregationMethod.ROTS
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)
df_rots = pd.DataFrame(aggregated_scores.dict()).applymap(lambda x: round(x['all']['score'], 2))
scores_collection.update({'ROTS': df_rots})
df_rots

See below how each aggregation method impact the scores on for each time frame and scope combination

In [ ]:
pd.concat(scores_collection, axis=0)